# rice CV



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd
import numpy as np
# import tensorflow as tf
import pathlib
from PIL import Image

In [ ]:
cur_path=pathlib.Path().absolute()
tr_xs_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\Train_Dev\training') ) #may use match
tr_ys_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\Train_Dev\train_labels') )

tr_ys_fls=[ i.stem for i in tr_ys_dir.iterdir() ]
dt_sample_num=0 #29 crazy small size #1 large size data #28 normal small, third num boxes, but quite close to 2nd num
df_ys= pd.read_csv(  tr_ys_dir.joinpath(tr_ys_fls[dt_sample_num]+'.csv') , names=['x','y']  )
#print(df_ys)

print('df shape:',df_ys.shape)

In [ ]:
img= Image.open( tr_xs_dir.joinpath(tr_ys_fls[dt_sample_num]+'.JPG') )
img.size

image 29 有4007pts，最多可能估5000較保險
image 是(1728, 2304, 3), (2000, 3000, 3) 兩種size

以下是把單純的點出水稻位置問題擴展成RCNN系列的region proposal 問題

In [ ]:
showBox=True
showPoint=False
showGrid=False
img_sample= mpimg.imread( tr_xs_dir.joinpath(tr_ys_fls[dt_sample_num]+'.JPG') )

print('img shape:',img_sample.shape)

In [ ]:
def get_pts(cen_pts,sq_width):
    anch_pts= cen_pts-sq_width//2
    end_pts= cen_pts+sq_width//2
    edgs_lengths=end_pts-anch_pts
    return anch_pts, end_pts, edgs_lengths

In [ ]:
fig, ax = plt.subplots( figsize=[6.4*4, 4.8*4] )

sq_width=60 #60 for No0, #26 for No28
n_h, n_w,n_c=img_sample.shape
print('ori image shape:', img_sample.shape)
expd_edge_width=sq_width//2
expd_edge_width=0

expd_img=np.zeros([n_h+expd_edge_width*2,n_w+expd_edge_width*2,n_c], dtype=np.uint8)
expd_img[expd_edge_width:expd_edge_width+n_h, expd_edge_width:expd_edge_width+n_w, :]=img_sample

img_sample=expd_img
n_h, n_w,n_c=img_sample.shape
print('expand image shape:', img_sample.shape)

ax.imshow( img_sample )
if showPoint:
    ax.scatter(df_ys['x']+expd_edge_width,df_ys['y']+expd_edge_width, 2,'r')


anch_pts, end_pts, edgs_lengths= get_pts(df_ys.values,sq_width)
anch_pts, end_pts= anch_pts+expd_edge_width, end_pts+expd_edge_width

# draw bounding box
if showBox:
    for i in range(df_ys.shape[0]):
        ax.add_patch( patches.Rectangle(anch_pts[i],edgs_lengths[i][0],edgs_lengths[i][1],
                                    edgecolor = 'blue',
                                    fill=False) )

    
    
pic_sp_width=sq_width//2* 7 #multi the grid nuber of decision network
#pic_sp_width=#nw//(n_w//pic_sp_width)

if showGrid:
    plt.hlines([i*pic_sp_width for i in range(n_h//pic_sp_width+1)],0,n_w)
    #plt.hlines([i*pic_sp_width for i in range(n_h//pic_sp_width+1)],0,n_w)

    plt.vlines([i*pic_sp_width for i in range(n_w//pic_sp_width+1)],0,n_h)
    #plt.vlines([i*pic_sp_width for i in range(n_w//pic_sp_width+1)],0,n_h)

# 自動pred box width的算法
開發自動pred box width的算法，圖中的9個點與其相鄰點距離的平均

In [ ]:
def find_n_nearest_pts(df_ys, pt_sample, n, x_name='x', y_name='y'):
    df_ys_find=df_ys.copy()
    df_ys_find['l2_square'] = (df_ys[x_name]-pt_sample[0])**2+(df_ys[y_name]-pt_sample[1])**2
    pt_n2=df_ys_find.sort_values('l2_square').iloc[1:1+n,:] # n nearest points
    pt_n2['distance']=np.sqrt(pt_n2['l2_square'])
    pt_n2.drop('l2_square', inplace=True, axis=1)
    return pt_n2

In [ ]:
fig=plt.figure( figsize=[6.4*4, 4.8*4] )
ax = fig.add_subplot(1, 1, 1)

xv,yv = np.meshgrid( range(n_w//4,n_w,n_w//4), range(n_h//4,n_h,n_h//4) )
xv=xv.flatten()
yv=yv.flatten()
pt_sample=[]
for i in range(xv.size):
    pt=find_n_nearest_pts(df_ys, [xv[i],yv[i]],1 )[['x','y']]
    pt_sample.append(pt)

pt_sample = pd.concat(pt_sample)
# pt_sample.iloc[0,:]=[ 889,1522 ] #ill case img28
print(pt_sample)

pts_nearest=[]
for i in pt_sample.iterrows():
    nearest_pt = find_n_nearest_pts(df_ys, i[1].values, 1)
    pts_nearest.append(nearest_pt)
pts_nearest=pd.concat(pts_nearest)
print( pts_nearest )

pred_width=int((np.average( pts_nearest['distance'].values )) *1)+1 #0.8
print( 'pred_width:',pred_width )


plt.scatter( pt_sample['x']+expd_edge_width, pt_sample['y']+expd_edge_width, 30, 'r')
plt.scatter( pts_nearest['x']+expd_edge_width, pts_nearest['y']+expd_edge_width, 30, 'y')
plt.imshow(img_sample )

anch_pts, end_pts, edgs_lengths= get_pts(df_ys.values,pred_width)
anch_pts, end_pts= anch_pts+expd_edge_width, end_pts+expd_edge_width

# draw bounding box
if showBox:
    for i in range(df_ys.shape[0]):
        ax.add_patch( patches.Rectangle(anch_pts[i],edgs_lengths[i][0],edgs_lengths[i][1],
                                    edgecolor = 'blue',
                                    fill=False) )

plt.show()

google object detection api 提供的現成model 最大就是1024x1024
這算的上是這個空拍圖辨視的極限，再小就很難框了。對於第28張圖，這甚至是不太ok的

# create tf record
now, let's create tf record data

the code following is base on [ models/research/object_detection/g3doc/using_your_own_dataset.md ](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md), thought the code is not up to date

## training

In [ ]:
def showBoxAndImage(img, anch_pts, end_pts):
    plt.figure(figsize=[6.4*1, 4.8*1])
    plt.imshow( img )
    ax = plt.gca()
    
    edgs_lengths = end_pts-anch_pts

    # draw bounding box
    for i in range(anch_pts.shape[0]):
        ax.add_patch( patches.Rectangle(anch_pts[i],edgs_lengths[i][0],edgs_lengths[i][1],
                                    edgecolor = 'blue',
                                    fill=False) )

    plt.show(file_path)
def readImgBytes(img_path):
    fd = open( img_path, mode='rb')
    img_str = fd.read()
    fd.close()
    return img_str

In [ ]:
def get_object_detection_data( file_path, label_path ):
    '''
    file_path=img_path, label_path should be pathlib object
    '''
    df_ys= pd.read_csv( label_path, names=['x','y']  )
    filename=file_path.name.encode() #should be bytes
    encoded_image_data = readImgBytes(file_path)
    img=Image.open(io.BytesIO(encoded_image_data))
    width, height = img.size
    image_format =file_path.suffix[1:].encode()
    
    xv,yv = np.meshgrid( range(width//4,width,width//4), range(height//4,height,height//4) )
    xv=xv.flatten()
    yv=yv.flatten()
    pt_sample=[]
    for i in range(xv.size):
        pt=find_n_nearest_pts(df_ys, [xv[i],yv[i]],1 )[['x','y']]
        pt_sample.append(pt)

    pt_sample = pd.concat(pt_sample)
    #print(pt_sample)

    pts_nearest=[]
    for i in pt_sample.iterrows():
        nearest_pt = find_n_nearest_pts(df_ys, i[1].values, 1)
        pts_nearest.append(nearest_pt)
    pts_nearest=pd.concat(pts_nearest)
    #print( pts_nearest )

    pred_width=int((np.average( pts_nearest['distance'].values )) *1)+1 #0.8
    #print( 'pred_width:',pred_width )
    anch_pts, end_pts, edgs_lengths= get_pts(df_ys.values, pred_width)
    
    xmins= anch_pts[:,0]/width
    xmaxs= end_pts[:,0]/width
    ymins= anch_pts[:,1]/height
    ymaxs= end_pts[:,1]/height
    classes_text= [b'rice']*df_ys.shape[0] #b'' mean encode
    classes= [1]*df_ys.shape[0] #class
    
    return height, width, filename, encoded_image_data, image_format, xmins, xmaxs, ymins, ymaxs, classes_text, classes

In [ ]:
import tensorflow as tf

from object_detection.utils import dataset_util
import io

#already deprecate in tf2 --> Module: tf.compat.v1.flags
# if realy need flag, use absl-py
#flags = tf.app.flags

# from absl import app
# from absl import flags

# flags.DEFINE_string('output_path', str(cur_path.parent.joinpath(r'data')), 'Path to output TFRecord')
# FLAGS = flags.FLAGS

In [ ]:


# def main(_):
#writer = tf.python_io.TFRecordWriter(FLAGS.output_path) #tf.compat.v1.python_io.TFRecordWriter(filename)
#https://github.com/tensorflow/models/issues/4794#issuecomment-497315185
cur_path=pathlib.Path().absolute()
tr_xs_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\Train_Dev\training') ) #may use match
tr_ys_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\Train_Dev\train_labels') )
tr_ys_fls=[ i.stem for i in tr_ys_dir.iterdir() ]
writer=tf.io.TFRecordWriter( str(cur_path.parent.joinpath(r'data/riceOdTrain.tfrecord')) ) #input output path
# print(writer)

# TODO(user): Write code to read in your dataset to examples variable

for fl_name in tr_ys_fls:
    print('fl_name:',fl_name)
    label_path= tr_ys_dir.joinpath( fl_name+'.csv')
    file_path= tr_xs_dir.joinpath( fl_name+'.JPG')
    height, width, filename, encoded_image_data, image_format, xmins, xmaxs, ymins, ymaxs, classes_text, classes=\
    get_object_detection_data( file_path, label_path )
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename), #source of the original image
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    writer.write(tf_example.SerializeToString())
    
    xmins*=width
    xmaxs*=width
    ymins*=height
    ymaxs*=height
    img = Image.open(io.BytesIO(encoded_image_data))
    anch_pts= np.concatenate((xmins[:,np.newaxis], ymins[:,np.newaxis]), axis=1)
    end_pts= np.concatenate((xmaxs[:,np.newaxis], ymaxs[:,np.newaxis]), axis=1)
    #showBoxAndImage(img, anch_pts, end_pts)
    
    print('============================end=================================\n')

writer.close()
print('finish training tfrecord file make')
# for example in examples:
#     tf_example = create_tf_example(example)
#     writer.write(tf_example.SerializeToString())

# writer.close()


# if __name__ == '__main__':
#     tf.app.run()
#     app.run()

## eval dataset

In [ ]:
cur_path=pathlib.Path().absolute()
tr_xs_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\sample_data\ori_image') ) #may use match
tr_ys_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\sample_data\csv_data') )
tr_ys_fls=[ i.stem for i in tr_ys_dir.iterdir() ]
writer=tf.io.TFRecordWriter( str(cur_path.parent.joinpath(r'data/riceOdEval.tfrecord')) ) #input output path
# print(writer)

# TODO(user): Write code to read in your dataset to examples variable

for fl_name in tr_ys_fls:
    print('fl_name:',fl_name)
    label_path= tr_ys_dir.joinpath( fl_name+'.csv')
    file_path= tr_xs_dir.joinpath( fl_name+'_new.jpg')
    height, width, filename, encoded_image_data, image_format, xmins, xmaxs, ymins, ymaxs, classes_text, classes=\
    get_object_detection_data( file_path, label_path )
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    writer.write(tf_example.SerializeToString())
    
    xmins*=width
    xmaxs*=width
    ymins*=height
    ymaxs*=height
    img = Image.open(io.BytesIO(encoded_image_data))
    anch_pts= np.concatenate((xmins[:,np.newaxis], ymins[:,np.newaxis]), axis=1)
    end_pts= np.concatenate((xmaxs[:,np.newaxis], ymaxs[:,np.newaxis]), axis=1)
    #showBoxAndImage(img, anch_pts, end_pts)
    
    print('============================end=================================\n')

writer.close()
print('finish eval tfrecord file make')

## development dataset
google object detection api 沒有doc (g3doc)說 inference 的tfrecord 要給什麼東西，google之後也沒有，我只能盡量猜一下

更正，inference 就是直接輸照片進去就ok 了，不用任何的 tfrecord

In [ ]:
# def get_eval_data( file_path ):
#     '''
#     file_path=img_path, label_path should be pathlib object
#     '''
#     filename=file_path.name.encode() #should be bytes
#     encoded_image_data = readImgBytes(file_path)
#     img=Image.open(io.BytesIO(encoded_image_data))
#     width, height = img.size
#     image_format =file_path.suffix[1:].encode()
    
#     return height, width, filename, encoded_image_data, image_format

In [ ]:
# cur_path=pathlib.Path().absolute()
# tr_xs_dir= cur_path.parent.joinpath( pathlib.PureWindowsPath( r'data\Train_Dev\development') ) #may use match
# tr_xs_fls=[ i.stem for i in tr_xs_dir.iterdir() ]
# writer= tf.io.TFRecordWriter( str(cur_path.parent.joinpath(r'data/riceOdDvlp.tfrecord')) ) #output path of input file
# # print(writer)

# # TODO(user): Write code to read in your dataset to examples variable

# for fl_name in tr_xs_fls:
#     print('fl_name:',fl_name)
#     file_path= tr_xs_dir.joinpath( fl_name+'.JPG')
#     height, width, filename, encoded_image_data, image_format= get_eval_data( file_path )
    
#     tf_example = tf.train.Example(features=tf.train.Features(feature={
#       'image/height': dataset_util.int64_feature(height),
#       'image/width': dataset_util.int64_feature(width),
#       'image/filename': dataset_util.bytes_feature(filename),
#       'image/source_id': dataset_util.bytes_feature(filename),
#       'image/encoded': dataset_util.bytes_feature(encoded_image_data),
#       'image/format': dataset_util.bytes_feature(image_format),
#     }))
#     writer.write(tf_example.SerializeToString())
#     print('============================end=================================\n')

# writer.close()
# print('finish evaluation tfrecord file make')